In [2]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from lazyqsar.agnostic import LazyBinaryClassifier
import pandas as pd
import numpy as np
import lazyqsar
import pickle
import h5py
import tqdm
import sys
import os

alpha = 0

# Define root directory
# root = os.path.dirname(os.path.abspath(__file__))
root = '.'

# Load pickle
pathogen_code, file, column = pickle.load(open(os.path.join(root, "..", "tmp", "models_to_train.pkl"), "rb"))[alpha]

# Load data
sys.stderr.write(str(pathogen_code) + "\n" + str(file) + " -- " + str(column))
sys.stderr.write("\n\n")
sys.stderr.flush()
data = pd.read_csv(os.path.join(root, "..", "output", pathogen_code, "datasets", file))
X = data['canonical_smiles'].astype(str).tolist()[:500]
Y = data['bin'].astype(np.int8).to_numpy()[:500]

# Define stratified 5 fold CV
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Load descriptors
sys.stderr.write("Loading descriptors...")
sys.stderr.write("\n\n")
sys.stderr.flush()
with h5py.File(os.path.join(root, "..", "output", pathogen_code, "descriptors.h5"), "r") as f:
    SMILES = f['SMILES'][:]
    X_Morgan = f['X_Morgan'][:]

# Define dict mapping smiles to morgan fingerprints
SMILES_TO_MORGAN = {
    smiles.decode("utf-8"): fp
    for (chembl_id, smiles), fp in zip(SMILES, X_Morgan)}

# For each split
AUROCS = []
for c, (train, test) in enumerate(skf.split(X, Y)):
    
    sys.stderr.write(f"Training split {c}...")
    sys.stderr.write("\n")
    sys.stderr.flush()

    # Get train/test indices
    X_train, Y_train = [X[i] for i in train], Y[train]
    X_test, Y_test = [X[i] for i in test], Y[test]

    # Get Morgan Fingerprints and train model
    X_train = np.array([SMILES_TO_MORGAN[smi] for smi in X_train])
    X_test  = np.array([SMILES_TO_MORGAN[smi] for smi in X_test])
    model = LazyBinaryClassifier(mode="fast")
    model.fit(X=X_train, y=Y_train)

    # Store results
    auroc = round(roc_auc_score(Y_test, model.predict_proba(X_test)[:, 1]), 3)
    AUROCS.append(auroc)
    sys.stderr.write(f"AUROC: {auroc}")
    sys.stderr.write("\n\n")
    sys.stderr.flush()

    break

# Train full model
X = np.array([SMILES_TO_MORGAN[smi] for smi in X])
model = LazyBinaryClassifier(mode="fast")
model.fit(X=X, y=Y)

# Save model
model.save(os.path.join(root, "..", "output", pathogen_code, "models", file.replace(".csv.gz", ".zip")))

# Save CV results
with open(os.path.join(root, "..", "output", pathogen_code, "models", file.replace('.gz', '')), "w") as outfile:
    outfile.write(",".join([str(i) for i in AUROCS]))

mtuberculosis
CHEMBL4649948_PERCENTEFFECT_%_perc_1.csv.gz -- bin

Loading descriptors...

Training split 0...


15:41:06 INFO     Dataset summary:                                                                                 
                    • Total samples: 400                                                                           
                    • Positive samples: 400                                                                        
                    • Negative samples: 0                                                                          
                    • Max samples per partition: 10000                                                             
                    • Min samples per partition: 30                                                                
                    • Positive proportion: 1.00

         INFO     Original positive samples: 400, total samples: 400

         INFO     Maximum samples: 10000

         DEBUG    Sampling 400 positive and 0 negative samples from 400 total samples.

         INFO     Unique sampled indices matrix shape: (1, 400)

IndexError: index 1 is out of bounds for axis 1 with size 1